In [12]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/AI/dataset/Energy_Power_Weather.xlsx", sheet_name=1)

Mounted at /content/drive


# 1.Tiền xử lý dữ liệu

In [15]:
# Chuyển cột 'Timestamp' sang kiểu datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Lấy chỉ thời gian và định dạng thành chuỗi '00:00:00'
df['Time'] = df['Timestamp'].dt.strftime('%H:%M:%S')

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Tạo cột mới cho giờ
df['Hour'] = df['Timestamp'].dt.hour

df['Date'] = df['Timestamp'].dt.date

df['YearMonth'] = df['Timestamp'].dt.to_period('M').astype(str)

df

,Timestamp,Watt,TempMin,TempMax,Weather,Time,Hour,Date,YearMonth
0,2019-01-28 00:00:00,0.0,18,23,Patchy rain possible,00:00:00,0,2019-01-28,2019-01
1,2019-01-28 00:15:00,0.0,18,23,Patchy rain possible,00:15:00,0,2019-01-28,2019-01
2,2019-01-28 00:30:00,0.0,18,23,Patchy rain possible,00:30:00,0,2019-01-28,2019-01
3,2019-01-28 00:45:00,0.0,18,23,Patchy rain possible,00:45:00,0,2019-01-28,2019-01
4,2019-01-28 01:00:00,0.0,18,23,Patchy rain possible,01:00:00,1,2019-01-28,2019-01
...,...,...,...,...,...,...,...,...,...
168283,2023-11-25 22:45:00,0.0,21,24,Light rain shower,22:45:00,22,2023-11-25,2023-11
168284,2023-11-25 23:00:00,0.0,21,24,Light rain shower,23:00:00,23,2023-11-25,2023-11
168285,2023-11-25 23:15:00,0.0,21,24,Light rain shower,23:15:00,23,2023-11-25,2023-11
168286,2023-11-25 23:30:00,0.0,21,24,Light rain shower,23:30:00,23,2023-11-25,2023-11


In [22]:
df_tem = df.groupby(["Date", "Weather"]).agg({'Watt': 'sum', 'TempMin': 'mean', 'TempMax': 'mean'}).reset_index()
df_tem = df_tem.drop(columns="Date", axis=1)
df_tem

,Weather,Watt,TempMin,TempMax
0,Patchy rain possible,8972.480,18.0,23.0
1,Patchy rain possible,25845.085,20.0,25.0
2,Patchy rain possible,36149.643,21.0,24.0
3,Sunny,65131.089,21.0,25.0
4,Moderate rain at times,36141.824,19.0,23.0
...,...,...,...,...
1748,Patchy rain possible,45215.285,22.0,26.0
1749,Light rain shower,19460.444,20.0,25.0
1750,Light rain shower,56680.677,23.0,25.0
1751,Patchy rain possible,56748.063,22.0,26.0


In [23]:
data_dummies = pd.get_dummies(df_tem, drop_first=True)
# Chuyển đổi kiểu dữ liệu của tất cả các giá trị trong DataFrame data_dummies thành float
data_dummies = data_dummies.astype(float)
data_dummies

,Watt,TempMin,TempMax,Weather_Heavy rain at times,Weather_Light rain shower,Weather_Mist,Weather_Moderate or heavy rain shower,Weather_Moderate rain at times,Weather_Overcast,Weather_Partly cloudy,Weather_Patchy light drizzle,Weather_Patchy light rain with thunder,Weather_Patchy rain possible,Weather_Sunny,Weather_Thundery outbreaks possible,Weather_Torrential rain shower
0,8972.480,18.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,25845.085,20.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,36149.643,21.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,65131.089,21.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,36141.824,19.0,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,45215.285,22.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1749,19460.444,20.0,25.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1750,56680.677,23.0,25.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1751,56748.063,22.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# 2.Tạo dữ liệu

In [24]:
from sklearn.model_selection import train_test_split

X = []
y = []

n = 14
m = 7

dataframe = data_dummies.values
for i in range(len(dataframe) - (m + n)):
    X.append(dataframe[i:i+n, 1:])
    y.append(dataframe[i+n:i+n+m, 0])
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

(1732, 14, 15) (1732, 7)
(1385, 14, 15)


# 3.Huấn luyện

### 3.1. LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError


def build_lstm(X_train, y_train):
    n_samples, timesteps, n_feature = X_train.shape
    model = Sequential()
    model.add(LSTM(200, activation="tanh", input_shape=(timesteps, n_feature)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer='adam', metrics=[RootMeanSquaredError(), "mae"])
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return model

md = build_lstm(X_train, y_train)

In [27]:
print(md.evaluate(X_test, y_test, verbose=0))
print(md.predict(X_test, verbose=0)[:7])
print(y_test[0])

[365513312.0, 19118.40234375, 15310.7294921875]
[[56264.156]
 [42883.094]
 [32466.273]
 [51831.39 ]
 [61256.52 ]
 [55838.824]
 [61256.676]]
[41383.527 13233.268  2603.151  1839.83   2294.786  6128.297  3675.964]


### 3.2 CNN-LSTM

In [30]:
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import Flatten
from keras.layers import RepeatVector
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError
from keras.layers import Dropout

def build_cnn_lstm(X_train, y_train):
    n_samples, time_step, n_feature = X_train.shape
    md = Sequential()
    md.add(Conv1D(64, kernel_size=3, activation="relu", input_shape=(time_step, n_feature)))
    md.add(MaxPool1D(2))
    md.add(Conv1D(128, kernel_size=3, activation="relu"))
    md.add(MaxPool1D(2))
    md.add(Dropout(0.2))
    md.add(Flatten())
    md.add(RepeatVector(time_step))
    md.add(LSTM(200, activation="tanh"))
    md.add(Dropout(0.2))
    md.add(Dense(100, activation='relu'))
    md.add(Dense(1))
    md.compile(loss="mse", optimizer="adam", metrics=[RootMeanSquaredError(), "mae"])
    md.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return md

cnn_lstm = build_cnn_lstm(X_train, y_train)
print(cnn_lstm.evaluate(X_test, y_test, verbose=0))
print(cnn_lstm.predict(X_test, verbose=0)[:7])
print(y_test[0])

[347265312.0, 18635.056640625, 14670.0634765625]
[[31538.824]
 [27193.31 ]
 [26796.014]
 [63335.39 ]
 [61860.21 ]
 [63767.55 ]
 [63340.492]]
[41383.527 13233.268  2603.151  1839.83   2294.786  6128.297  3675.964]
